In [1]:
# coding: utf-8
import os
try:
    from pathlib import Path
except:    
    from pathlib2 import Path
import argparse
import sys
import itertools
import geopandas as gpd
from datetime import datetime, timedelta
import copy
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter
import numpy as np
import pandas as pd
from scipy import optimize
import fire
from tqdm import tqdm

#for debugging within notebook
try:
    from IPython.core.debugger import set_trace
except:
    pass

# for python 2/3 compatibility
try:
    xrange
except NameError:
    xrange = range    
try:
    get_ipython().magic('matplotlib inline')
except:
    pass

class WaterstandStatistiek(object):
    
    class __errors__(object):
        """
        error class met verschillende foutmeldingen
        """
        @staticmethod
        def fileNotFound():
            raise IOError('HIS-bestand bestaat niet. Is je pad goed?')        
        @staticmethod
        def metadataNotSet():
            raise AttributeError('Metadata is niet bekend. Zet met sobek.LeesMetadata(hisFileNaam)')
        @staticmethod
        def metadataError():
            raise AttributeError('Metadata kon tijdens de functie LeesMetadata() niet bepaald worden')
        @staticmethod
        def administratieError():            
            raise AttributeError('Administratieblok van his-file kan niet worden uitgelezen')
        @staticmethod
        def locatieNotFound():            
            raise AttributeError('Locatie niet gevonden. Is het een bestaande locatie?')
        @staticmethod
        def variabeleNotFound():            
            raise AttributeError('Parameter niet gevonden. Is het een bestaande parameter?')
        @staticmethod
        def jaarmaxError():
            raise ValueError('Voor de MultiWaardenArray functie kan alleen `year` of `none` gebruikt worden als voor de jaarmax_as parameter')
        @staticmethod
        def gewogenGemiddeldeError():
            raise ValueError('Kan gewogen gemiddelde niet bepalen. Vereiste is minimaal 2 punten aan de linker en rechterzijde van de gekozen T')            
    
    def __init__(self):
        """
        SobekResultaten class. Class met methodes om een .his bestand 
        uit te lezen.
        WaterstandStatistiek class. Class met methodes om een GumbleFit te bepalen
        """
        # initatie van lege immutable objects
        # zie: https://stackoverflow.com/a/22526544/2459096
        # python2.7: obj = argparse.Namespace()
        # python3.x: obj = types.SimpleNamespace()
        self.hisFile = argparse.Namespace()
        self.hisFile.tijdstapInfo = argparse.Namespace()
        self.hisFile.variabeleInfo = argparse.Namespace()
        self.hisFile.locatieInfo = argparse.Namespace()
        
        self.stats = argparse.Namespace()
        self.stats.plotposities = argparse.Namespace()
        self.stats.stap1 = argparse.Namespace()
        self.stats.stap2 = argparse.Namespace()
        self.stats.stap3 = argparse.Namespace()
        self.stats.stap4 = argparse.Namespace()         
        
    def _leesHeader(self, f):
        """
        Lees de header van het bestand

        Parameters
        ----------
        f : BufferedReader    
            Leesbaar ruw IO object

        Returns
        -------
        header : array
            De offset van de locatie in bytes    
        """
        header = []
        for i in range(4):    
            header.append(f.read(40).decode('windows-1252'))  
        return header           
        
    def _infoFromHeader(self,tijdstapInfo):
        """
        Lees informatie uit de header

        Parameters
        ----------
        tijdstapInfo : str
            De locatie in string format

        Returns
        -------
        beginDate : datetime 
            De begindatum T0, waarop de andere datetime gebasseerd zijn
        timeStepInterval : str
            De eenheid waarin de stapgrote van de datetime objecten berekend zijn, 
            dit kan zijn s [seconds], m [minutes], h[hours], d[days]
        timeStepFactor : int
            Factor welke op de stapgrote toegepast moet worden
        """

        items = str(tijdstapInfo).split(' ')    
        beginDate = datetime.strptime(items[1],'%Y.%m.%d')

        scu = items[-1].replace(')','').replace("'",'')
        timeStepInterval =  scu[(len(scu) - 1)]
        timeStepFactor = int(scu[0:len(scu)-1])

        return beginDate, timeStepInterval, timeStepFactor    

    def _leesAdmin(self, f):
        """
        Lees het administratie blok
        """
        self.hisFile.header = self._leesHeader(f)

        # lees info van header
        self.hisFile.tijdstapInfo.beginDate, self.hisFile.tijdstapInfo.timeStepInterval, self.hisFile.tijdstapInfo.timeStepFactor = self._infoFromHeader(self.hisFile.header[3])        

        # lees aantal variablelen
        self.hisFile.variabeleInfo.numVar = np.fromfile(f,np.int,1)[0]    

        # lees aantal locaties
        self.hisFile.locatieInfo.numLoc = np.fromfile(f,np.int,1)[0]    

        # lees variabelen
        self.hisFile.variabeleInfo.variabelen = []
        for i in range(self.hisFile.variabeleInfo.numVar):
            self.hisFile.variabeleInfo.variabelen.append(f.read(20).decode('windows-1252'))    

        # Lees locaties
        self.hisFile.locatieInfo.id = []
        self.hisFile.locatieInfo.locaties = []
        for i in range(self.hisFile.locatieInfo.numLoc):            
            self.hisFile.locatieInfo.id.append(np.fromfile(f,np.int,1)[0])
            self.hisFile.locatieInfo.locaties.append(f.read(20).decode('windows-1252').rstrip())
            #self.hisFile.locatieInfo.locaties.append(locatieInfo)

        # lees tijdstappen
        self.hisFile.headerSize = f.tell()
        self.hisFile.locatieInfo.numTime = int((self.hisFile.hisFileSize - self.hisFile.headerSize) /
                      (self.hisFile.locatieInfo.numLoc * self.hisFile.variabeleInfo.numVar + 1) / 4)

        aantalBytesPerStap = int((self.hisFile.hisFileSize - self.hisFile.headerSize) / self.hisFile.locatieInfo.numTime)

        byteNr = self.hisFile.headerSize

        self.hisFile.tijdstapInfo.moments = []
        
        self.hisFile.tijdstapInfo.offset = []
        for i in range(self.hisFile.locatieInfo.numTime):
            f.seek(byteNr)
            moment = np.fromfile(f,np.int,1)[0]

            if self.hisFile.tijdstapInfo.timeStepInterval == 's':
                self.hisFile.tijdstapInfo.moments.append(self.hisFile.tijdstapInfo.beginDate + timedelta(seconds = (float(moment) * self.hisFile.tijdstapInfo.timeStepFactor)))
            elif self.hisFile.tijdstapInfo.timeStepInterval == 'm':
                self.hisFile.tijdstapInfo.moments.append(self.hisFile.tijdstapInfo.beginDate + timedelta(minutes = (float(moment) * self.hisFile.tijdstapInfo.timeStepFactor)))
            elif self.hisFile.tijdstapInfo.timeStepInterval == 'h':
                self.hisFile.tijdstapInfo.moments.append(self.hisFile.tijdstapInfo.beginDate + timedelta(hours= (float(moment) * self.hisFile.tijdstapInfo.timeStepFactor)))
            elif self.hisFile.tijdstapInfo.timeStepInterval == 'd':
                self.hisFile.tijdstapInfo.moments.append(self.hisFile.tijdstapInfo.beginDate + timedelta(days= (float(moment) * self.hisFile.tijdstapInfo.timeStepFactor)))
            self.hisFile.tijdstapInfo.offset.append(f.tell())
            byteNr += aantalBytesPerStap
        self.hisFile.tijdstapInfo.N = np.array(self.hisFile.tijdstapInfo.moments).max().year - np.array(self.hisFile.tijdstapInfo.moments).min().year + 1
        return True

    def _locOffset(self, loc, var):
        """
        Krijg de offset van de variabele

        Parameters
        ----------
        loc : string    
            De locatie in kwestie
        var : string
            De variabele in kwestie        

        Returns
        -------
        locOffset : int
            De offset van de locatie in bytes
        """
        #  Zoek de index van de variabele
        try:
            varFound = self.hisFile.variabeleInfo.variabelen.index(var)
        except :
            self.__errors__.variabeleNotFound()            

        # Zoek de index/id van de locatie
        try:
            # removed dictionary
            # locFound = next((item for item in self.hisFile.locatieInfo.locaties if item['locatie'] == loc))
            locFound = self.hisFile.locatieInfo.locaties.index(loc)
        except :
            self.__errors__.locatieNotFound()

        locOffset = locFound * self.hisFile.variabeleInfo.numVar * 4 + varFound * 4

        return locOffset   
    
    def KrijgLokaties(self):
        """
        Krijg de locaties beschikbaar in de his-file. Vereiste is dat de LeesMetadata functie is uitgelezen
        
        Returns
        -------
        locaties : list
            De locaties welke bekend zijn binnen het his-bestand
        """
        locaties = self.hisFile.locatieInfo.locaties
        return locaties
    
    def KrijgParameters(self):
        """
        Krijg de parameters beschikbaar in de his-file. Vereiste is dat de LeesMetadata functie is uitgelezen
        
        Returns
        -------
        parameters : list
            De parameters welke bekend zijn binnen het his-bestand
        """
        parameters = self.hisFile.variabeleInfo.variabelen
        return parameters

    def KrijgTijdstappen(self):
        """
        Krijg de tijdstappen beschikbaar in de his-file. Vereiste is dat de LeesMetadata functie is uitgelezen
        
        Returns
        -------
        tijdstappen : list
            De tijdstappen welke bekend zijn binnen het his-bestand
        """
        tijdstappen = self.hisFile.tijdstapInfo.moments
        return tijdstappen    

    def LeesMetadata(self,hisFileNaam):
        """
        Open het bestand

        Parameters
        ----------
        hisFileNaam : str
            Volledige pad van het bestand

        Returns
        -------
        Str : 'Metadata Ingelezen'
        
        """
        self.hisFile.metaDataIngelezen = False
        myHisFile = Path(hisFileNaam)
        try:
            p = myHisFile.resolve()
        except:
            # doesn't exist
            self.__errors__.fileNotFound()
        else:
            # exists    
            self.hisFile.hisFileName = str(p)            
            # set length filesize (bytes)
            statInfo = os.stat(his_file)
            self.hisFile.hisFileSize = statInfo.st_size
            #f = open(str(p), "rb")
            with open(self.hisFile.hisFileName, "rb") as f:
                try:
                    self._leesAdmin(f)        
                except:
                    self.__errors__.administratieError()
            
        self.hisFile.metaDataIngelezen = True
        return 'Metadata Ingelezen'
    
    def SelectPeriodeWaardenArray(self, df, startMMdd=(1,1), endMMdd=(12,31), jaarmax_as='date'):
        """
        Selecteer op basis van een DataFrame de gebeurtenissen binnen een bepaalde periode (bijv. een groeiseizoen).
        Waarbij er de mogelijkheid is om de gebeurtenissen binnen deze periode te groeperen.
        Pas op: in de `EnkeleWaardenArray` en `MultiWaardenArray` wordt deze functie intern ook aangeroepen.
        
        Parameters
        ----------
        df : pandas.DataFrame
            DataFrame met datetime als index
        startMMdd : tuple
            Tuple in het formaat (M,d). Bepaling van de start datum van de periode. Genoemde datum is inclusief
        endMMdd : tuple
            Tuple in het formaat (M,d). Bepaling van de eind datum van de periode. Genoemde datum is inclusief
        jaarmax_as : str
            Mogelijkheden om de DataFrame te groeperen op basis van jaar om het jaarmaxima te bepalen. 
            Het jaarmaxima wordt bepaald nadat de slice van de jaarlijkse periode is toegepast. 
            Keuze bestaat uit:            
            'date' - bepaalt de jaarlijkse maxima en geeft de maximale waarde terug met de exacte datum van deze gebeurtenis
            'year' - bepaalt de jaarlijkse maxima en geeft de maximale waarde terug met het jaar van de gebeurtenis
            'none' - retourneert alle gebeurtenissen in elk jaar
            
        Returns
        -------
        df : pandas.DataFrame
            DataFrame met datetime als index
            
        Examples
        --------
        Voor bepaling van het groeiseizoen:
        Bijvoorbeeld het jaarlijks groeseizoen loopt van 15 april tot en met 11 oktober
        startMMdd = (4,15)
        endMMdd = (10,11)
        
        betekent, slice elk jaar in april na de 15e inclusief tot aan oktober voor de 11e inclusief
        
        Voor bepaling van het jaarmaxima:
        groupby = 'date'
                    A
        2012-10-06  1501
        2013-04-22  1534
        2014-04-18  1591

        groupby = 'year'
                    A
        2012        1501
        2013        1534
        2014        1591
        """
        self.startMM, self.startdd = startMMdd
        self.endMM, self.enddd = endMMdd    
        
        # https://stackoverflow.com/a/45996897/2459096
        # maak een month_day dataframe van de MultiYear DataFrame
        month_day = pd.concat([
                        df.index.to_series().dt.month, 
                        df.index.to_series().dt.day
                    ], axis=1).apply(tuple, axis=1)        
        
        # selecteer alleen de gebeurtenissen binnen een jaarlijks terugkerende periode 
        df = df[(month_day >= (self.startMM, self.startdd)) & (month_day <= (self.endMM, self.enddd))]

        if jaarmax_as=='year':
            # groupby year en selecteer de max. 
            # hetvolgende retourneert alleen jaren + max, willen van elk jaar de datum + max
            df = df.groupby(df.index.year).max()

        elif jaarmax_as=='date':
            # groupy year en selecteer de max. Returned volle datums waar de max van dat jaar was 
            key = df.columns.levels[1][0]
            level = df.columns.names[1]
            slice_col = df.columns.levels[0][0]

            idx = df.xs(key=key, level=level, axis=1).groupby([df.index.year])[slice_col].transform(max) == df.xs(key=key, level=level, axis=1)[slice_col]
            df = df[idx]
        
        elif jaarmax_as=='none':
            #doe niets
            pass
    
        return df

    def EnkeleWaardenArray(self, locatie, parameter, startMMdd=(1,1), endMMdd=(12,31), jaarmax_as='date'):
        """
        Lees de waarden van een enkele variabele op een enkele locatie

        Parameters
        ----------
        locatie : string    
            De locatie in kwestie (alleen de eerste 21 karakters, anders cutoff)
        parameter : string
            De variabele in kwestie (alleen de eerste 21 karakters, anders cutoff)
        startMMdd : tuple
            Tuple in het formaat (M,d). Bepaling van de start datum van de periode. Genoemde datum is inclusief
        endMMdd : tuple
            Tuple in het formaat (M,d). Bepaling van de eind datum van de periode. Genoemde datum is inclusief
        jaarmax_as : str
            Mogelijkheden om de DataFrame te groeperen op basis van jaar om het jaarmaxima te bepalen. 
            Het jaarmaxima wordt bepaald nadat de slice van de jaarlijkse periode is toegepast. 
            Keuze bestaat uit:            
            'date' - bepaalt de jaarlijkse maxima en geeft de maximale waarde terug met de exacte datum van deze gebeurtenis
            'year' - bepaalt de jaarlijkse maxima en geeft de maximale waarde terug met het jaar van de gebeurtenis
            'none' - retourneert alle gebeurtenissen in elk jaar            

        Returns
        -------
        df : DataFrame
            DataFrame met een multicolumn van variabele en locaties met datetime index en 
            de bijbehorende waarden
            
        Examples
        -----------
        locatie = '1'
        parameter = 'Waterlevel maximum (mNAP)'
        
        Intern worden alle inputs afgekort op [0:20] en ziet de tool het als:
        locatie = '1'
        parameter = 'Waterlevel maximum ('

        Voor bepaling van het groeiseizoen:
        Bijvoorbeeld het jaarlijks groeseizoen loopt van 15 april tot en met 11 oktober
        startMMdd = (4,15)
        endMMdd = (10,11)
        
        betekent, slice elk jaar in april na de 15e inclusief tot aan oktober voor de 11e inclusief
        
        Voor bepaling van het jaarmaxima:
        groupby = 'date'
                    A
        2012-10-06  1501
        2013-04-22  1534
        2014-04-18  1591

        groupby = 'year'
                    A
        2012        1501
        2013        1534
        2014        1591
        """

        if not hasattr(self.hisFile, 'metaDataIngelezen'):
            self.__errors__.metadataNotSet()
        
        if self.hisFile.metaDataIngelezen == False:
            self.__errors__.metadataError()
        
        loc = locatie[0:20]
        var = parameter[0:20]
        
        with open(self.hisFile.hisFileName, "rb") as f: 
            varLocOffset = self._locOffset(loc, var)
            values= []

            for i in range(self.hisFile.locatieInfo.numTime):
                offset = self.hisFile.tijdstapInfo.offset[i] + varLocOffset                
                seek = f.seek(offset)
                values.append(np.fromfile(f,np.float32,1)[0])

        # maak dataframe
        df = pd.DataFrame(data=values, index=self.hisFile.tijdstapInfo.moments, columns=[(loc,var)])
        df.columns = pd.MultiIndex.from_tuples(df.columns, names=['locatie','parameter'])
        
        df = self.SelectPeriodeWaardenArray(df, startMMdd=startMMdd, endMMdd=endMMdd, jaarmax_as=jaarmax_as)

        return df#.T.squeeze()
    
    def MultiWaardenArary(self, locaties, parameters, startMMdd=(1,1), endMMdd=(12,31), 
                          jaarmax_as='year', drop_lege_jaren=True):
        """
        Lees de waarden van meerdere variabelen op meerdere locaties

        Parameters
        ----------
        locaties : list    
            Een list van de locaties (type: str) in kwestie (alleen de eerste 21 karakters, anders cutoff) 
        parameters : list
            Een list van de parameters in kwestie (alleen de eerste 21 karakters, anders cutoff) 
        startMMdd : tuple
            Tuple in het formaat (M,d). Bepaling van de start datum van de periode. Genoemde datum is inclusief
        endMMdd : tuple
            Tuple in het formaat (M,d). Bepaling van de eind datum van de periode. Genoemde datum is inclusief
        jaarmax_as : str
            Mogelijkheden om de DataFrame te groeperen op basis van jaar om het jaarmaxima te bepalen. 
            Het jaarmaxima wordt bepaald nadat de slice van de jaarlijkse periode is toegepast. 
            Keuze bestaat uit:            
            'year' - bepaalt de jaarlijkse maxima en geeft de maximale waarde terug met het jaar van de gebeurtenis
            'none' - retourneert alle gebeurtenissen in elk jaar  
        drop_lege_jaren : boolean
            Mogelijkheid om de jaren te verwijderen welke geen gebeurtenissen bevatten voor de gegeven selectie. 
            Wanneer dit als `True` is geselecteerd zal het jaar alleen gedropt worden als deze niet bestaat voor 
            alle locaties/parameters in de selectie

        Returns
        -------
        df : DataFrame
            DataFrame met een multicolumn van variabele en locaties met datetime index en 
            de bijbehorende waarden
            
        Examples
        -----------
        locaties = ['1','6']
        parameters = ['Waterlevel maximum (mNAP)']
        
        Intern worden alle inputs afgekort op [0:20] en ziet de tool het als:
        locaties = ['1','6']
        parameters = ['Waterlevel maximum (']
        """
        # error checking
        if jaarmax_as not in ('year', 'none'):
            self.__errors__.jaarmaxError()
            
        # define empty dataframe
        clmns = pd.MultiIndex.from_product([parameters,locaties], names=['parameters','locaties'])
        
        # voor een MultiWaardenArray wordt alleen een jaar meegenomen als index
        # het hangt van een seizoen filter af of er gebeurtenissen voor dat jaar zijn. Bepaal dit eerst
        loc0 = self.hisFile.locatieInfo.locaties[0]
        par0 = self.hisFile.variabeleInfo.variabelen[0]
        idx = self.EnkeleWaardenArray(loc0, par0, jaarmax_as=jaarmax_as).index
        
        self.hisFile.tijdstapInfo.N
        
        df_full = pd.DataFrame(index=idx, columns=clmns)
        df_full.index.name='date'
        for ix in clmns:
            df = self.EnkeleWaardenArray(locatie=ix[1], parameter=ix[0], 
                                         startMMdd=startMMdd, endMMdd=endMMdd, jaarmax_as=jaarmax_as)
            df_full.loc[:,(ix[0],ix[1])] = df.T.squeeze()
        
        # bepaling of lege jaren gedropt moeten worden
        if drop_lege_jaren:
            df_full.dropna(axis=0, how='all', inplace=True)
        
        return df_full
                 
    def _best_fit_slope_and_intercept(self,xs,ys):
        m = (((xs.mean()*ys.mean()) - (xs*ys).mean()) / ((xs.mean()*xs.mean()) - (xs*xs).mean()))
        b= ys.mean() - m*xs.mean()
        return m, b

    def _squared_error(self,ys_orig,ys_line):
        return sum((ys_line - ys_orig) * (ys_line - ys_orig))

    def _coefficient_of_determination(self,ys_orig,ys_line):
        y_mean_line = [ys_orig.mean() for y in ys_orig]
        squared_error_regr = self._squared_error(ys_orig, ys_line)
        squared_error_y_mean = self._squared_error(ys_orig, y_mean_line)
        return 1 - (squared_error_regr/squared_error_y_mean)

    def _r_squared(self,xs,ys):
        """
        Bepaalt de r**2 waarde op basis van twee input arrays

        Parameters
        ----------
        xs : numpy.array
            array met orginele waarden
        ys : numpy.array
            array met afgeleide waarden

        Returns
        -------
        r_squared : float
            de afgeleide r**2 waarde
        """
        m, b = self._best_fit_slope_and_intercept(xs,ys)
        regression_line = [(m*x)+b for x in xs]

        r_squared = self._coefficient_of_determination(ys,regression_line)
        return r_squared        
        
    def _igumbelFunc(self, arr, Td, t):
        """
        Gumbel Functie
        
        Parameters
        ----------
        arr : numpy.array
            array met gesorteerde waterstanden
        Td : 
        
        Returns
        -------
        gumbelFuncFunc : numpy.array
            Gumbel Functie
        """
        a,d = arr
        self.stats.plotposities.gumbelFunc = (float(-1.0)* float(a)*np.log(1.0/t * float(Td))+float(d))
        return self.stats.plotposities.gumbelFunc
    
    def _igumbelFuncFit(self, arr, h, Td, t):
        """
        Gumbel Functie
        
        Parameters
        ----------
        arr : numpy.array
            array met gesorteerde waterstanden
        h : int?
        
        Td : int?
        
        T : int?
        
        Returns
        -------
        gumbelFuncFit : numpy.array
            Fitted Gumbel Functie
        """        
        a,d = arr
        self.stats.plotposities.gumbelFuncFit = h-(float(-1.0)* float(a)*np.log(1.0/t * float(Td))+float(d))        
        return self.stats.plotposities.gumbelFuncFit
    
    def _gewogenGemiddelde4TOI(self, Tarray, WSarray, TOI):
        """
        Functie om het gewogen gemiddelde te krijgen op basis van de vier dichtsbijzijnde waarden rondom de T of interest

        Parameters
        ----------
        Tarray : numpy.array
            Array met de terugkeertijden, 
        WSarray : numpy.array
            Array met de parameterwaarden, bijvoorbeeld waterstanden
        TOI : int
            Waarde met de terugkeertijd of interest, bijvoorbeeld 10 voor T=10

        Returns
        -------

        """
        # get the index of the closest value to T10 on its left side, 
        # T is sorted in descending order so reverse array first to get ascending order
        cl = self._find_closest(Tarray[::-1], TOI)    

        # get the indices of the four closest points
        # [..]  *     *      *      *   |     *     *  [..]
        #           cl-2   cl-1    cl  T10  cl+1  cl+2
        #                  ----    --       ----  ----
        try:
            indices = np.array([cl-1,cl,cl+1,cl+2])

            WSarray_sel  = np.take(WSarray[::-1], indices)
            Tarray_sel   = np.take(Tarray[::-1], indices) 
        except:
            self.__errors__.gewogenGemiddeldeError()

        # inverse distance weighting
        weights = 1 / abs(Tarray_sel - TOI)
        weights /= weights.sum()    

        # weight_cl-1 * ws_cl-1  +  weight_cl * ws_cl  +  weight_cl+1 * ws_cl+1  +  weight_cl+2 * ws_cl+2
        WSarray_TOI = np.dot(weights, WSarray_sel)
        return WSarray_TOI, WSarray_sel, Tarray_sel

    def _find_closest(self, Tarray, TOI):
        # T must be sorted (in ascending order)
        idx = Tarray.searchsorted(TOI)
        idx = np.clip(idx, 1, len(Tarray)-1)
        left = Tarray[idx-1]
        right = Tarray[idx]
        idx -= TOI - left < right - TOI
        return idx    
        
    def _calcFi_T(self, len_arr,N, Ggi=0.44, GgN=0.12):
        """
        Bepaal plotpositie jaarmaximum als kans        

        Parameters
        ----------
        len_arr : int
            aantal gebeurtenissen
        N : int
            aantal jaren waarover de Gumble fit bepaald moet worden
        Ggi : float
            Gringgorten plotposities i (standaard: 0.44)
        GgN : float
            Gringgorten plotposities N (standaard: 0.12)

        Returns
        -------
        T : np.array
            array van terugkeertijden
        Fijaar : np.array
            array van plotposities jaarmaxima
            
        Literature
        ----------
        Voor bepaling Gringorten coefficienten zie bijv: http://glossary.ametsoc.org/wiki/Gringorten_plotting_position
        """
        Fi = []        
        T = []
        for i in range(len_arr):
            i += 1
            f = float(1 - ((i - Ggi) / (N+GgN)))
            Fi.append(f)
        Fijaar = (np.log(Fi) )* -1
        T = 1 / (Fijaar)
        # set set self
        self.stats.plotposities.Fijaar = Fijaar
        self.stats.plotposities.T = T
        return self.stats.plotposities.T, self.stats.plotposities.Fijaar
    
    def PlotFiguur(self,out_folder):
        """
        plot figuur
        
        Parameters
        ----------
        ID : str
            Locatienaam
        
        ## STEP1 Gumbel Fit voor T25,T50,T100 voor alle buien
        S1_vAmin : int
            venster array minimum
        S1_vAmax : int
            venster array maximum
        S1_T : ndarray
            Terugkeertijden van jaarmaxima gebeurtenissen genomen over gehele jaar
        S1_ws_srt : ndarray
            Waterstanden welke horen bij de terugkeertijden van jaarmaxima gebeurtenissen genomen over gehele jaar
        S1_GumbelT : ndarray
            Terugkeertijden welke horen bij de Gumbel fit (eg, 25,50,100)
        S1_GumbelWS : ndarray
            Waterstanden welke horen bij de terugkeertijden welke horen bij de Gumbel fit (eg, 25,50,100)
        S1_GumbelWS_line : ndarray
            Waterstanden welke horen bij de terugkeertijden welke horen bij de Gumbel fit vallend binnend het venster
        S1_r_squared : float
            r**2 tussen de Gumbel fit en gebeurtenenissed binnen je venster
        
        ## STEP2 Gumbel Fit voor T10 voor alleen zomerbuien
        S2_T : ndarray
            Terugkeertijden van jaarmaxima gebeurtenissen genomen over het groeiseizoen
        S2_ws_srt : ndarray
            Waterstanden welke horen bij de terugkeertijden van jaarmaxima gebeurtenissen genomen over het groeiseizoen
        
        ## STEP4 Gewogen gemiddelde voor T10 voor alleen de zomerbuien
        S4_Tarray_sel_jaar : ndarray
            Terugkeertijden van jaarmaxima gebeurtenissen genomen over het groeiseizoen welke meegenomen zijn voor bepaling gewogen gemiddeld
        S4_WSarray_sel_jaar : ndarray
            Waterstanden welke horen bij terugkeertijden van jaarmaxima gebeurtenissen welke meegenomen zijn voor bepaling gewogen gemiddeld
        TOI : int
            Terugkeertijd of interest voor bepaling van het gewogen gemiddelde (normaal gesproken 10)
        S4_WSarray_TOI_jaar : float
            Waterstand welke hoort bij de TOI
        
        Returns
        -------
        Maakt figuur en slaat deze in de folder waar het script gerund wordt.
        """

        ID = self.stats.stap1.ID
        S1_vAmin = self.stats.stap1.vAmin
        S1_vAmax = self.stats.stap1.vAmax
        S1_T = self.stats.stap1.T
        S1_ws_srt = self.stats.stap1.ws_srt
        S1_GumbelT = self.stats.stap1.GumbelT
        S1_GumbelWS = self.stats.stap1.GumbelWS
        S1_GumbelWS_line = self.stats.stap1.GumbelWS_line
        S1_r_squared = self.stats.stap1.r_squared
        
        S2_T = self.stats.stap2.T
        S2_ws_srt = self.stats.stap2.ws_srt
        
        S4_Tarray_sel_jaar = self.stats.stap4.Tarray_sel_jaar
        S4_WSarray_sel_jaar = self.stats.stap4.WSarray_sel_jaar
        TOI = self.stats.stap4.TOI
        S4_WSarray_TOI_jaar = self.stats.stap4.WSarray_TOI_jaar  
        
        
        # Voor het figuur mag de TOI niet geplot worden in de gumbel fit. 
        # Kijk of TOI binnen de GumbelT bestaan en verwijder dit punt als zo.
        if TOI in S1_GumbelT:            
            idx = S1_GumbelT.index(TOI)
            S1_GumbelT = np.delete(S1_GumbelT, idx)
            S1_GumbelWS = np.delete(S1_GumbelWS, idx)       
        
        fig=plt.figure(figsize=(8,6))
        ax = fig.add_subplot(111)
        ax.set_xscale('log')

        # plot jaarmaxima gebeurtenissen over gehele jaar ## OPEN CIRKELS
        ax.plot(S1_T, S1_ws_srt, fillstyle='none', label='waterstanden (jaarmaxima)',
                color='#2E589F', linestyle='', marker='o', markeredgewidth=2, markersize=10, markerfacecoloralt='gray')

        # plot maxima gebeurtenissen over de zomerbuien ## DICHTE CIRKELS
        ax.plot(S2_T, S2_ws_srt, fillstyle='full', label='waterstanden (maxima groeiseizoen)',
               color='cornflowerblue', linestyle='', marker='o', markersize=8, markerfacecoloralt='gray', alpha=0.75)

        # plot gumble fit voor T25, T50, T100 in lijn en punt
        ax.plot(S1_T[S1_vAmin:S1_vAmax],S1_GumbelWS_line, '-', color='#3BBB75', label='Gumbel fit (venster)')
        ax.scatter(S1_GumbelT, S1_GumbelWS,s=100, marker='s', facecolors='#3BBB75', zorder=10)

        # plot gewogen gemiddelde voor T10 op basis van zomerbuien in punt ## HALF-GEVULDE CIRKELS
        ax.scatter(TOI, S4_WSarray_TOI_jaar,s=100, marker='s',facecolors='#3BBB75', zorder=10,label='waterstand op T10, T25, T50 en T100')
        ax.plot(S4_Tarray_sel_jaar,S4_WSarray_sel_jaar, fillstyle='right', label='gebeurtenissen voor bepaling T'+str(TOI),
               color='#F5AC1B', linestyle='none', marker='o', markersize=8, markerfacecoloralt='white')
    
        # plot r**2 rechtsbovenin
        ax.text(0.975, 0.975, '$r^2$: '+str(np.round(S1_r_squared,2)),
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes)
        
        # axes settings
        ax.set_xlim(1,300)
        Ymin = min(S1_ws_srt) - 0.05
        Ymax = max(S1_ws_srt) + 0.25
        ax.set_ylim(Ymin,Ymax)
        ax.set_ylabel('Waterstand (m+NAP)')
        ax.set_xlabel('Terugkeertijd (jaren)')
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        ax.grid(True,which='both', axis='both', linestyle='-', color= '0.75', zorder=0)

        # legend settings
        legend = plt.legend(loc='upper left',prop={'size':12})
        legend.get_frame().set_facecolor('white')
        legend.get_frame().set_alpha(1)
        legend.get_frame().set_linewidth(1)
        legend.get_frame().set_edgecolor('#8B8B8B')

        # table settings
        col_labels=['Terugkeertijd','Waterstand (m+NAP)']        
        table_vals = pd.DataFrame([['T'+str(i) for i in S1_GumbelT],S1_GumbelWS.round(decimals=2).tolist()]).T.values.tolist()        
        table_vals = [['T'+str(TOI), str(round(S4_WSarray_TOI_jaar,2))]] + table_vals

        # the rectangle is where I want to place the table
        the_table = ax.table(cellText=table_vals,
                          colWidths = [0.4]*2,
                          colLabels=col_labels,
                          loc='center right',bbox=[1.01, 0.0, 0.5, 0.5])                

        plt.title('locatie: '+ID)
        plt.tight_layout()
        out_folder_png = os.path.join(out_folder,'png')
        out_folder_png_file = os.path.join(out_folder_png, 'waterstandstatistiek_loc_'+ID+'.png')
        plt.savefig(out_folder_png_file, bbox_inches='tight', dpi=10, pad_inches=0.2) 
        plt.clf()
        plt.close(fig)
    
    def _gewogenGemiddelde(self, df_enkel, N=109,TOI=10):
        """
        """
        # lengte van array van waterstanden
        Nws = df_enkel.size        
        T, Fijaar = self._calcFi_T(Nws,N)
        
        # kijk eerst of de input dataframe een multi-column heeft en reduceerd indien ja
        try:
            df_enkel = df_enkel.iloc[:,0]
        except:
            pass

        if len(df_enkel.name) == 2:
            enkel_locatie = df_enkel.name[0]
            enkel_parameter = df_enkel.name[1]
        else: 
            enkel_locatie = enkel_parameter = 'niet gedefinieerd'          
        
        df_enkel_sorted = df_enkel.sort_values(ascending=False)        
        
        # bepaal gewogen gemiddelde voor parameter waarde voor terugkeertijd of interest op basis van 4 dichtsbijzijnde waarden
        WSarray_TOI, WSarray_sel, Tarray_sel = self._gewogenGemiddelde4TOI(T, df_enkel_sorted.values, TOI)
        return WSarray_TOI, WSarray_sel, Tarray_sel
    
    def _enkeleGumbelFit(self, df_enkel, N, vensterArray=[0,9], GumbelT=[25,50,100.0], Ggi=0.44, GgN=0.12):
        """
        TOI : int
            Terugkeertijd of interest
            
        N : int
            Pas op: de bepaling voor over hoeveel jaren het aantal plotposities berekend moet worden, 
            moet gedaan worden op basis van het aantal mogelijke jaren in de gebeurtenissen reeks. 
            Bepaal dit op een gebeurtenissenreeks alvorens een periode filter is toegepast. 
        """        

        # lengte van array van waterstanden
        Nws = df_enkel.size        
        T, Fijaar = self._calcFi_T(Nws,N, Ggi, GgN)        
        

        # kijk eerst of de input dataframe een multi-column heeft en reduceerd indien ja
        try:
            df_enkel = df_enkel.iloc[:,0]
        except:
            pass

        if len(df_enkel.name) == 2:
            enkel_locatie = df_enkel.name[0]
            enkel_parameter = df_enkel.name[1]
        else: 
            enkel_locatie = enkel_parameter = 'niet gedefinieerd'   
            
        # get min/max van venster array
        vAmin = vensterArray[0]
        vAmax = vensterArray[1]
        
        df_enkel_sorted = df_enkel.sort_values(ascending=False)                
        
        # bepaal de terugkeertijden waarden en afgeleiden op basis van venster array voor Gumbel fit
        dT = np.log(T[vAmin]) - np.log(T[vAmax])
        dh = df_enkel_sorted.iloc[vAmin] - df_enkel_sorted.iloc[vAmax]
        dTpunt = -np.log(T[vAmax])
        dhpunt = (dh/dT)*dTpunt
        a0 = [dh/dT, df_enkel_sorted.iloc[vAmax]+dhpunt]
        
        # apply de gumbel fit functie
        a_out, cov_x, infodict, msg, flag = optimize.leastsq(self._igumbelFuncFit, a0, args = (df_enkel_sorted.iloc[vAmin:vAmax], 1, T[vAmin:vAmax]), full_output=True)
        # a_out holds the paramters of the fitted line [a and b]
        a = a_out[0]
        b = a_out[1]
        # get the locations on the fit corresponding to the required return periods
        GumbelWS = self._igumbelFunc(a_out, 1, np.array(GumbelT))
        # get the locations on the fit corresponding to the slice on all Ts using the vensterArray
        GumbelWS_lineVensterArray = self._igumbelFunc(a_out, 1, T[vAmin:vAmax])
        #set_trace()
        
        xs = df_enkel_sorted.iloc[vAmin:vAmax]
        ys = GumbelWS_lineVensterArray
        r_squared = self._r_squared(xs,ys)
        
        # schrijf weg naar gumbel dataframe
        T_columns = ['T'+str(int(item)) for item in GumbelT]
        df_gumbel = pd.DataFrame(index=[enkel_locatie], columns=T_columns+['solution'])
        df_gumbel.index.name='locatie'

        df_gumbel.loc[enkel_locatie][T_columns] = GumbelWS
        if flag > 4:
            df_gumbel.loc[enkel_locatie]['solution'] = 'no solution'
        else:
            df_gumbel.loc[enkel_locatie]['solution'] = 'optimization succesful'  
        
#         if plotFiguur==True:
#             self.plotFiguur(vAmin,vAmax, df_enkel_sorted.values, T, GumbelT, GumbelWS, enkel_locatie)            
            
        return (vAmin,vAmax, df_enkel_sorted.values, T, GumbelT, GumbelWS, enkel_locatie, a,b, GumbelWS_lineVensterArray,r_squared), df_gumbel

    def AfleidingParameters(self, df_enkel, N, vensterArray, GumbelT, TOI, startMMdd=(1,1), endMMdd=(12,31), 
                            jaarmax_as='date', Ggi=0.44, GgN=0.12):
        """
        
        """
        # STEP 1 Gumbel Fit voor geselecteerde T's [eg. T10,T25,T50,T100] voor alle buien
        S1_param4fig, S1_df_gumbel = self._enkeleGumbelFit(df_enkel, N=N, vensterArray=vensterArray, GumbelT=GumbelT,Ggi=Ggi,GgN=GgN)
        self.stats.stap1.vAmin = S1_param4fig[0]
        self.stats.stap1.vAmax = S1_param4fig[1]
        self.stats.stap1.ws_srt = S1_param4fig[2]
        self.stats.stap1.T = S1_param4fig[3]
        self.stats.stap1.GumbelT = S1_param4fig[4]
        self.stats.stap1.GumbelWS = S1_param4fig[5]
        self.stats.stap1.ID = S1_param4fig[6]
        self.stats.stap1.a = S1_param4fig[7]
        self.stats.stap1.b = S1_param4fig[8]
        self.stats.stap1.GumbelWS_line = S1_param4fig[9]
        self.stats.stap1.r_squared = S1_param4fig[10]
        
        # STEP 2 Gumbel Fit voor T10 voor alleen zomerbuien
        df_enkel_groei = self.SelectPeriodeWaardenArray(df_enkel, startMMdd=startMMdd, endMMdd=endMMdd, jaarmax_as=jaarmax_as)
        S2_param4fig, S2_df_gumbel = self._enkeleGumbelFit(df_enkel_groei, N=N, GumbelT=[TOI], vensterArray = vensterArray,Ggi=Ggi,GgN=GgN)
        self.stats.stap2.ws_srt = S2_param4fig[2]
        self.stats.stap2.T = S2_param4fig[3]
        self.stats.stap2.GumbelT = S2_param4fig[4]
        self.stats.stap2.GumbelWS = S2_param4fig[5]
        self.stats.stap2.startMMdd = startMMdd
        self.stats.stap2.endMMdd = endMMdd
        
        # STEP 3 Gewogen gemiddelde voor T10 voor alle buien
        self.stats.stap3.WSarray_TOI_jaar, self.stats.stap3.WSarray_sel_jaar, self.stats.stap3.Tarray_sel_jaar = self._gewogenGemiddelde(df_enkel, N=N, TOI=TOI)
        self.stats.stap3.TOI = TOI
        self.stats.stap3.startMMdd = startMMdd
        self.stats.stap3.endMMdd = endMMdd        

        # STEP 4 Gewogen gemiddelde voor T10 voor alleen de zomerbuien
        self.stats.stap4.WSarray_TOI_jaar, self.stats.stap4.WSarray_sel_jaar, self.stats.stap4.Tarray_sel_jaar = self._gewogenGemiddelde(df_enkel_groei, N=N, TOI=TOI)
        self.stats.stap4.TOI = TOI
        self.stats.stap4.startMMdd = startMMdd
        self.stats.stap4.endMMdd = endMMdd        
        return 'sucess'
    
    def EnsembleRunner(self, out_folder, his_file, shp_file, shp_key='nodeID', parameter='auto',
                       startMMdd=(5,15), endMMdd=(10,15), vensterArray=[0,10], GumbelT=[10,25,50,100], 
                       TOI=10, draw_plot=True, write_table=True):
        """
        Schrijf de afgeleide waterstanden naar een tabel structuur

        Input
        -----
        folder_path : Path
            schrijf csv weg binnen opgegeven folder
        locaties_df : geodataframe
            shapefile met nodeIDs als een kolom beschikbaar in het dataframe
        locaties_kolom : str
            naam van de kolom in de geodataframe met de nodeIDs
        decimals : int
            aantal decimalen voor afronding            

        Return
        ------
        csv_path : Path
            path naar weggeschreven csv bestand (table 1)
        shp_path : Path
            path naar weggeschreven shp bestand (table 2)
        """
        # create frame table
        self.LeesMetadata(his_file)
        self.stats.df_table1 = pd.DataFrame(columns=[shp_key,'H','T','seizoen','methode'])
        self.stats.df_table2 = pd.DataFrame(columns=[shp_key,'T10','T25','T50','T100','r^2','a','b'])
        
        locaties_df = gpd.read_file(shp_file)
        
        if parameter=='auto':
            parameter = self.hisFile.variabeleInfo.variabelen[0]

        # iterate locationlist and create item in table
        pbar = tqdm(locaties_df[shp_key])
        for locatie in pbar:
            pbar.set_description("Processing %s" % locatie)
            df_enkel = self.EnkeleWaardenArray(locatie, parameter, startMMdd=(1, 1), endMMdd=(12, 31), jaarmax_as='date')            
            self.AfleidingParameters(df_enkel, self.hisFile.tijdstapInfo.N, vensterArray, GumbelT, TOI, startMMdd, endMMdd, jaarmax_as='date')
            if draw_plot == True:
                self.PlotFiguur(out_folder)
            if write_table ==True:
                self.SchrijfTabellen(shp_key=shp_key)
        
        # join on input shapefile and write tables
        # save table 1
        self.stats.df_table1.to_csv(os.path.join(out_folder,'waterstandstatistiek_output_1.csv'),index=False)
        
        # prepare table 2 and save
        locaties_df = locaties_df.set_index(shp_key).join(self.stats.df_table2.set_index(shp_key)).reset_index()
        locaties_df.to_file(os.path.join(out_folder,'waterstandstatistiek_output_2.shp'), driver='ESRI Shapefile')
        self.stats.df_table2 = locaties_df

    def SchrijfTabellen(self,shp_key,decimals=3):
        """
        """
        table1_list = []
        table2_list = []
        # gewogen gemiddelde voor jaarmaxima alle buien (# STEP 3)
        table1_list.append({shp_key:self.stats.stap1.ID,
                   'H':round(self.stats.stap3.WSarray_TOI_jaar,decimals),
                   'T':self.stats.stap3.TOI,
                   'seizoen':'geheel jaar',
                   'methode':'gewogen gemiddelde'})

        # gewogen gemiddelde voor maxima zomerbuien (# STEP 4)
        table1_list.append({shp_key:self.stats.stap1.ID,
                   'H':round(self.stats.stap4.WSarray_TOI_jaar,decimals),
                   'T':self.stats.stap4.TOI,
                   'seizoen':'alleen zomer',
                   'methode':'gewogen gemiddelde'})

        # Gumbel fit voor enkel maxima zomerbuien (# STEP 2)
        table1_list.append({shp_key:self.stats.stap1.ID,
                   'H':round(self.stats.stap2.GumbelWS[0],decimals),
                   'T':self.stats.stap2.GumbelT[0],
                   'seizoen':'alleen zomer',
                   'methode':'Gumbel fit'})

        # Gumbel fit voor jaarmaxima alle buien (# STEP 1)
        for idx, T in enumerate(self.stats.stap1.GumbelT):
            item = {shp_key:self.stats.stap1.ID,
                    'H':round(self.stats.stap1.GumbelWS[idx],decimals),
                    'T':T,
                    'seizoen':'geheel jaar',
                    'methode':'Gumbel fit'}    
            table1_list.append(item)

        # append to dataframe 1
        self.stats.df_table1 = self.stats.df_table1.append(table1_list, ignore_index=True)

        table2_list.append({shp_key:self.stats.stap1.ID,
                            'T10':self.stats.df_table1[(self.stats.df_table1['T'] == 10) & 
                                            (self.stats.df_table1['methode'] == 'gewogen gemiddelde') & 
                                            (self.stats.df_table1['seizoen'] == 'geheel jaar') &
                                            (self.stats.df_table1[shp_key] == self.stats.stap1.ID)]['H'].values[0],
                            'T25':self.stats.df_table1[(self.stats.df_table1['T'] == 25) & 
                                            (self.stats.df_table1[shp_key] == self.stats.stap1.ID)]['H'].values[0],
                            'T50':self.stats.df_table1[(self.stats.df_table1['T'] == 50) & 
                                            (self.stats.df_table1[shp_key] == self.stats.stap1.ID)]['H'].values[0],
                            'T100':self.stats.df_table1[(self.stats.df_table1['T'] == 100) & 
                                            (self.stats.df_table1[shp_key] == self.stats.stap1.ID)]['H'].values[0],
                            'r^2':round(self.stats.stap1.r_squared,decimals),
                            'a':round(self.stats.stap1.a,decimals),
                            'b':round(self.stats.stap1.b,decimals)})    

        # append to dataframe 2
        self.stats.df_table2 = self.stats.df_table2.append(table2_list, ignore_index=True)     
        return 

if __name__ == '__main__':
    try:
        # for jupyter notebooks
        fire.Fire(WaterstandStatistiek)       
    except:
        # for command line requests
        ws = WaterstandStatistiek() 
        pass

Fire trace:
1. Initial component
2. Instantiated class "WaterstandStatistiek"
3. ('Could not consume arg:', '-f')

Type:           WaterstandStatistiek
String form:    <__main__.WaterstandStatistiek object at 0x104D6DB0>
Init docstring: SobekResultaten class. Class met methodes om een .his bestand 
uit te lezen.
WaterstandStatistiek class. Class met methodes om een GumbleFit te bepalen

Usage:          __main__.py 
                __main__.py AfleidingParameters
                __main__.py EnkeleWaardenArray
                __main__.py EnsembleRunner
                __main__.py KrijgLokaties
                __main__.py KrijgParameters
                __main__.py KrijgTijdstappen
                __main__.py LeesMetadata
                __main__.py MultiWaardenArary
                __main__.py PlotFiguur
                __main__.py SchrijfTabellen
                __main__.py SelectPeriodeWaardenArray
                __main__.py hisFile
                __main__.py stats


In [2]:
his_file = r'D:\Projects\Pr\3651.10\Werkfolder\hkvpy\tests\test_data//CALCPNT.HIS'
shp_file = r'D:\Projects\Pr\3651.10\Werkfolder\hkvpy\tests\test_data//input_locaties.shp'
out_folder = r'D:\Projects\Pr\3651.10\Werkfolder\hkvpy\tests\output'
# input parameters voor afleiden T10,25,50en 100
vensterArray = [0,10] 
GumbelT = [10,25,50,100]
# T welke gebruikt wordt voor het gewogen gemiddelde
TOI = 10
# periode groeiseizoen
startMMdd = (5,15)
endMMdd = (10,15)

In [3]:
ws.EnsembleRunner(out_folder=out_folder,his_file=his_file,shp_file=shp_file,shp_key='nodeID',
                  startMMdd=(5,15),endMMdd=(10,15),vensterArray=[0,10],GumbelT=[10,25,50,100],TOI=10,
                  draw_plot=True, write_table=True)

Processing 26:   9%|██▌                         | 8/86 [00:04<00:38,  2.02it/s]C:\Python275\lib\site-packages\ipykernel\__main__.py:697: RuntimeWarning: invalid value encountered in log
C:\Python275\lib\site-packages\ipykernel\__main__.py:538: RuntimeWarning: invalid value encountered in double_scalars
C:\Python275\lib\site-packages\numpy\core\fromnumeric.py:57: RuntimeWarning: invalid value encountered in rint
  return getattr(obj, method)(*args, **kwds)
Processing 71: 100%|███████████████████████████| 86/86 [00:41<00:00,  2.10it/s]
